In [ ]:
#  Tell me about a situation where you solved a very complex problem.

# 1 Amazon Cart discount and scalable

Write code that will be used by a Shopping cart service to enforce rules on the order

eg.

Offer free 2 day shipping on orders > $35 if customer is not a prime member

Offer free 2 day shipping on all orders if customer is a prime member

Offer free 1 day shipping for order that are > $125

Offer free 2 hour shipping for prime customer that have > $25 and the items are grocery items

Make this extensible to add other rules in the future

Apply a 10% discount if an item has been marked for subscribe and save

In [ ]:
from typing import List

class Order:
    def __init__(self, total_amount: float, items: List[str], is_prime: bool, is_grocery: bool, subscribe_and_save: bool):
        self.total_amount = total_amount
        self.items = items
        self.is_prime = is_prime
        self.is_grocery = is_grocery
        self.subscribe_and_save = subscribe_and_save
        self.shipping_type = 'Standard'
        self.discount = 0.0


class Rule:
    def apply(self, order: Order):
        raise NotImplementedError

class FreeTwoDayShippingRule(Rule):
    def apply(self, order: Order):
        if order.is_prime or order.total_amount > 35:
            order.shipping_type = 'Free 2 Day Shipping'

class FreeOneDayShippingRule(Rule):
    def apply(self, order: Order):
        if order.total_amount > 125:
            order.shipping_type = 'Free 1 Day Shipping'

class FreeTwoHourShippingRule(Rule):
    def apply(self, order: Order):
        if order.is_prime and order.is_grocery and order.total_amount > 25:
            order.shipping_type = 'Free 2 Hour Shipping'

class SubscribeAndSaveDiscountRule(Rule):
    def apply(self, order: Order):
        if order.subscribe_and_save:
            order.discount = 0.10 * order.total_amount


class ShoppingCartService:
    def __init__(self):
        self.rules = []

    def add_rule(self, rule: Rule):
        self.rules.append(rule)
        rule.apply(order)


# Example usage:
order = Order(total_amount=150, items=['item1', 'item2'], is_prime=False, is_grocery=False, subscribe_and_save=True)

service = ShoppingCartService()

service.add_rule(FreeTwoDayShippingRule())
service.add_rule(FreeOneDayShippingRule())
service.add_rule(FreeTwoHourShippingRule())
service.add_rule(SubscribeAndSaveDiscountRule())

# service.apply_rules(order)

print(f"Shipping Type: {order.shipping_type}")
print(f"Discount: {order.discount}")
print(f"Total after Discount: {order.total_amount - order.discount}")

In [ ]:
# # 1 Amazon Cart discount and scalable (Selft)

# Write code that will be used by a Shopping cart service to enforce rules on the order

# eg.

# Offer free 2 day shipping on orders > $35 if customer is not a prime member

# Offer free 2 day shipping on all orders if customer is a prime member

# Offer free 1 day shipping for order that are > $125

# Offer free 2 hour shipping for prime customer that have > $25 and the items are grocery items

# Make this extensible to add other rules in the future

# Apply a 10% discount if an item has been marked for subscribe and save


class Order:

    def __init__( self, total, items,  is_groc, is_fav,is_prime=True,) -> None:
        
        self.total, self.items, self.is_prime, self.is_groc, self.is_fav = total, items, is_prime, is_groc, is_fav
        self.discount = 0
        self.deltype = 'Regular'


class Rule:
    def apply(self, order: Order):
        raise NotImplemented

class rule1(Rule):
    def apply(self, order: Order):
        if order.total > 35 or order.is_prime:
            order.deltype = 'two day'
        
class rule2(Rule):
    def apply(self, order):
        if order.total >125:
            order.deltype = 'one day'

class rule3(Rule):
    def apply(self, order):
        if order.total >25 and order.is_groc:
            order.deltype = 'one hour'


class rule4(Rule):
    def apply(self, order):
        if order.is_fav:
            order.discount = 0.1 * order.total

       
order = Order(total=35, items=['item1', 'item2'], is_groc = True, is_fav = True)

rule1().apply(order)
rule2().apply(order)
rule3().apply(order)
rule4().apply(order)

print(f"Order Total: {order.total}")
print(f"Items: {order.items}")
print(f"Is Prime Member: {order.is_prime}")
print(f"Is Grocery: {order.is_groc}")
print(f"Subscribe and Save: {order.is_fav}")
print(f"Shipping Type: {order.deltype}")
print(f"Discount: {order.discount}")
print(f"Total after Discount: {order.total - order.discount}")


In [ ]:
from abc import ABC, abstractmethod
from typing import List

class File:
    def __init__(self, name: str, extension: str, size: int):
        self.name = name
        self.extension = extension
        self.size = size

    def get_name(self) -> str:
        return self.name

    def get_extension(self) -> str:
        return self.extension

    def get_size(self) -> int:
        return self.size

class FileSystem:
    def __init__(self, name: str, is_directory: bool = False, sub_directories: List['FileSystem'] = None, files: List[File] = None):
        self.name = name
        self.is_directory = is_directory
        self.sub_directories = sub_directories if sub_directories else []
        self.files = files if files else []


class Filter(ABC):
    @abstractmethod
    def match(self, file: File) -> bool:
        pass

class NameFilter(Filter):
    def __init__(self, name: str):
        self.name = name

    def match(self, file: File) -> bool:
        return file.get_name() == self.name

class SizeFilter(Filter):
    def __init__(self, size: int, operator: str):
        self.size = size
        self.operator = operator

    def match(self, file: File) -> bool:
        return eval(str(file.get_size()) + str(self.operator) + str(self.size))

class ExtensionFilter(Filter):
    def __init__(self, extension: str):
        self.extension = extension

    def match(self, file: File) -> bool:
        return file.get_extension() == self.extension


class Search:
    def __init__(self, file_system: FileSystem, filters: List[Filter], condition: str = "AND"):
        self.file_system = file_system
        self.filters = filters
        self.condition = condition

    def check_conditions(self, file: File) -> bool:
        if self.condition == "AND":
            for f in self.filters:
                if not f.match(file):
                    return False
            return True
        
        elif self.condition == "OR":
            for f in self.filters:
                if f.match(file):
                    return True
            return False
        
        return False

    def find_files(self) -> List[str]:
        queue = [self.file_system]
        result = []

        # BFS
        while queue:
            node = queue.pop(0)
            for file in node.files:
                if self.check_conditions(file):
                    result.append(file.get_name())
            for sub_directory in node.sub_directories:
                queue.append(sub_directory)

        return result

if __name__ == "__main__":
    # Simulation
    f1 = File("abc", "txt", 10)
    f2 = File("cde", "txt", 20)
    f3 = File("def", "pdf", 30)
    f4 = File("ghi", "py", 5)
    f5 = File("uvw", "java", 10)

    directory_files = [f1, f2, f3, f4, f5]
    file_system = FileSystem("/", True, [], directory_files)

    # Example searches
    res = Search(file_system, [NameFilter("abc")]).find_files()
    print("Files with name 'abc':", res)

    res = Search(file_system, [SizeFilter(10, ">=")]).find_files()
    print("Files with size >= 10:", res)

    res = Search(file_system, [ExtensionFilter("java"), SizeFilter(10, ">=")], "OR").find_files()
    print("Files with extension 'java' OR size >= 10:", res)


In [ ]:
from abc import ABC, abstractmethod

class VehicleSize:
    MOTORCYCLE = 'Motorcycle'
    CAR = 'Car'
    BUS = 'Bus'

class Vehicle(ABC):
    def __init__(self, license_plate, size):
        self.license_plate = license_plate
        self.size = size

    @abstractmethod
    def can_fit_in_spot(self, spot):
        pass

class Motorcycle(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate, VehicleSize.MOTORCYCLE)

    def can_fit_in_spot(self, spot):
        return True  # A motorcycle can park anywhere

class Car(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate, VehicleSize.CAR)

    def can_fit_in_spot(self, spot):
        return spot.size in [VehicleSize.CAR, VehicleSize.BUS]

class Bus(Vehicle):
    def __init__(self, license_plate):
        super().__init__(license_plate, VehicleSize.BUS)

    def can_fit_in_spot(self, spot):
        return spot.size == VehicleSize.BUS

class ParkingSpot:
    def __init__(self, number, size):
        self.number = number
        self.size = size
        self.vehicle = None

    def park(self, vehicle):
        if self.is_available() and vehicle.can_fit_in_spot(self):
            self.vehicle = vehicle
            return True
        return False

    def remove_vehicle(self):
        self.vehicle = None

    def is_available(self):
        return self.vehicle is None

class ParkingLot:
    def __init__(self, num_spots):
        self.spots = [ParkingSpot(i, self.assign_size(i)) for i in range(num_spots)]

    def assign_size(self, index):
        if index % 3 == 0:
            return VehicleSize.BUS
        elif index % 2 == 0:
            return VehicleSize.CAR
        else:
            return VehicleSize.MOTORCYCLE

    def park_vehicle(self, vehicle):
        for spot in self.spots:
            if spot.park(vehicle):
                return True
        return False

    def remove_vehicle(self, vehicle):
        for spot in self.spots:
            if spot.vehicle == vehicle:
                spot.remove_vehicle()
                return True
        return False

# Example usage
parking_lot = ParkingLot(30)
car = Car("CAR123")
motorcycle = Motorcycle("MOTO456")
bus = Bus("BUS789")

print(parking_lot.park_vehicle(car))        # Attempts to park car
print(parking_lot.park_vehicle(motorcycle)) # Attempts to park motorcycle
print(parking_lot.park_vehicle(bus))        # Attempts to park bus

parking_lot.remove_vehicle(car)             # Remove the car from the spot


In [ ]:
class TicTacToe:
    def __init__(self, n):
        self.n = n
        self.board = [[0] * n for _ in range(n)]
        self.rowSum = [0] * n
        self.colSum = [0] * n
        self.diagSum = 0
        self.revDiagSum = 0

    def move(self, player, row, col):
        if not (0 <= row < self.n and 0 <= col < self.n):
            raise ValueError("Move out of board boundary!")
        if self.board[row][col] != 0:
            raise ValueError("Square is already occupied!")
        if player not in (1, 2):
            raise ValueError("Invalid player!")

        # Player 1 is +1, Player 2 is -1 (to easily calculate sums)
        value = 1 if player == 1 else -1
        self.board[row][col] = value

        # Update rowSum, colSum, and diagonals
        self.rowSum[row] += value
        self.colSum[col] += value
        if row == col:
            self.diagSum += value
        if row + col == self.n - 1:
            self.revDiagSum += value

        # Check if this move wins the game
        if (abs(self.rowSum[row]) == self.n or
            abs(self.colSum[col]) == self.n or
            abs(self.diagSum) == self.n or
            abs(self.revDiagSum) == self.n):
            return player

        return 0

# Example usage
game = TicTacToe(3)
print(game.move(2, 0, 0))  # Player 1 moves at (0, 0)
print(game.move(1, 0, 1))  # Player 2 moves at (0, 1)
print(game.move(2, 1, 1))  # Player 1 moves at (1, 1)
print(game.move(1, 0, 2))  # Player 2 moves at (0, 2)
print(game.move(2, 2, 2))  # Player 1 moves at (2, 2), potentially wins


In [22]:
class File:
    name = 'abc'

    def __init__(self,fname) -> None:
        print('called')
        self.fname = fname
        # self.name = 'ram'


    def open(self,flag):
        print('opend with flag')

class Jenga(File):
    pass

f1 = Jenga('first file')

called


In [23]:
f1.fname
f1.name
f1.open('yes')

opend with flag
